In [ ]:
### LLM API Examples
# This script demonstrates how to interact with a LLM API

First of all, ensure all the required libraries are installed

In [1]:
pip install google-genai streamlit

^C
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     -------------------- ------------------- 30.7/60.9 kB 1.4 MB/s eta 0:00:01
     ------------------------------- ------ 51.2/60.9 kB 871.5 kB/s eta 0:00:01
     -------------------------------------- 60.9/60.9 kB 649.9 kB/s eta 0:00:00
     ---------------------------------------- 0.0/44.3 kB ? eta -:--:--
     ---------------------------------------- 44.3/44.3 kB 2.3 MB/s eta 0:00:00
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/10.1 MB 1.4 MB/s eta 0:00:07
   ---------------------------------------- 0.1/10.1 MB 1.4 MB/s eta 0:00:07
   ---------------------------------------- 0.

Setup you API Key

In [ ]:
my_api_key = "YOUR_API_KEY_HERE"

#### Test API Calling

In [ ]:
from google import genai
from google.genai import types

# Initialize the client with your API key
client = genai.Client(api_key= my_api_key)

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Explain how AI works in a few words",
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=0) # Disables thinking, making the response faster but less accurate
    ),
)
print(response.text)

---

#### Ask Agent

In [ ]:
from google import genai
from google.genai import types

# Choose a model. E.g. “gemini-2.5-flash” or whichever you have access to. :contentReference[oaicite:1]{index=1}
MODEL = "gemini-2.5-flash"

def ask_agent(prompt: str) -> str:

    # Initialize the client with your API key
    client = genai.Client(api_key=my_api_key)

    response = client.models.generate_content(
        model=MODEL,
        contents=prompt
    )
    # The client returns a structure; here we extract the generated text
    return response.text or ""

def run_agent():
    print("Agent ready. Type 'exit' to quit.")
    while True:
        user = input("You: ")
        if user.strip().lower() == "exit":
            break
        reply = ask_agent(user)
        print("Agent:", reply)

if __name__ == "__main__":
    run_agent()

---